# Дякую за цікаву задачу=)


In [422]:
import pandas as pd
import numpy as np


from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import roc_auc_score
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

In [229]:
# load data

def load_data():
    X = pd.read_csv('tabular_data.csv', encoding='latin-1')
    hashed_feature = pd.read_csv('hashed_feature.csv', encoding='latin-1')
    y_train = pd.read_csv('train.csv', encoding='latin-1')
    y_test = pd.read_csv('test.csv', encoding='latin-1')
    return hashed_feature, X.drop(['feature_41','period','feature_0'], axis = 1), y_train.drop('id', axis = 1), y_test.drop('id', axis = 1)

In [230]:
hashed_feature, X, y_train, y_test = load_data()

In [240]:
X

,id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_39,feature_40,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,0,55.0,0.432017,0.000000,176.78,0.0,0.323712,391.127914,147.0,0.000000,...,0.00,0.00,0.000000,0.0,55.0,2.0,0.526552,145.0,133.28,0.0
1,0,110.0,0.397517,0.000000,315.42,0.0,0.316798,391.127914,135.0,0.000000,...,15.75,0.00,0.000000,0.0,110.0,1.0,0.481063,130.0,229.97,0.0
2,0,55.0,0.359440,0.000000,354.55,0.0,0.339188,391.127914,181.0,0.000000,...,0.00,0.07,0.000000,0.0,55.0,1.0,0.509598,180.0,231.78,0.0
3,0,55.0,0.285707,0.000000,229.98,0.0,0.415428,391.127914,142.0,0.000000,...,0.00,0.00,0.000000,0.0,55.0,0.0,0.680089,142.0,183.83,0.0
4,0,55.0,0.101487,444.730391,307.12,0.0,0.569670,391.127914,92.0,0.000000,...,26.75,0.95,20.014485,0.0,55.0,0.0,0.776175,85.0,155.83,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61267,5105,77.0,0.305150,0.000000,95.30,0.0,0.094381,221.850000,-44.0,0.000000,...,6.03,-0.00,0.000000,0.0,75.0,0.0,0.082128,43.0,83.97,0.0
61268,5105,77.0,0.305150,0.000000,61.87,0.0,0.105878,343.220000,52.0,0.000000,...,17.30,1.75,0.000000,0.0,75.0,0.0,0.077594,49.0,48.02,0.0
61269,5105,106.5,0.305150,0.000000,88.05,0.0,0.040169,259.330000,46.0,4676.517443,...,22.60,8.27,0.000000,0.0,99.0,1.0,0.109053,43.0,74.52,0.0
61270,5105,45.0,0.305150,0.000000,98.40,0.0,0.040810,222.000000,-39.0,0.000000,...,21.13,0.00,0.000000,0.0,0.0,0.0,0.101929,35.0,25.83,0.0


In [232]:
pd.crosstab(X['feature_25'], X.id)

id,0,1,2,3,4,5,6,7,8,9,...,5096,5097,5098,5099,5100,5101,5102,5103,5104,5105
feature_25,,,,,,,,,,,,,,,,,,,,,
11e623a37e87cf7995c466723ec99688d55cae8c,7,0,1,9,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
390c481a740bb1c12c57f33dc5263ced2ab11796,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aa2c96dacf00c451ef465f6115a45a20bccf1256,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,12,0,0
b204e1e3743cc314841070435211e25f4602b9fd,0,0,11,0,0,0,12,0,0,0,...,0,0,12,0,0,0,12,0,0,12
cf3cca1da7361bd988642600e76c4a3021be8ccf,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cf7413906c99be47067cb3fb6299959857710d77,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
d458ece2abeae803254c5d442b2e2a80f58a4153,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
f9ae10397ed62e79c644d06842bbcd92963a5927,4,12,0,3,12,12,0,12,11,12,...,12,12,0,12,12,12,0,0,12,0


In [233]:
X['feature_25'] = X['feature_25'].map({'11e623a37e87cf7995c466723ec99688d55cae8c': 1,
                                                        '390c481a740bb1c12c57f33dc5263ced2ab11796': 2,
                                                        'aa2c96dacf00c451ef465f6115a45a20bccf1256': 30,
                                                        'b204e1e3743cc314841070435211e25f4602b9fd': 33,                                                       
                                                        'cf3cca1da7361bd988642600e76c4a3021be8ccf': 34,                                                                                                            
                                                        'cf7413906c99be47067cb3fb6299959857710d77': 35,                                                      
                                                        'd458ece2abeae803254c5d442b2e2a80f58a4153': 36,                                                      
                                                        'f9ae10397ed62e79c644d06842bbcd92963a5927': 60,                                                      
                                                      })

In [234]:
#Fill NaN
for column in X.columns:
    X[column] = X[column].fillna(X[column].mean())

In [241]:
def sum_by_period(X):
    x_sum = X.drop(index = range(1,X.shape[0]))
    x_sum = x_sum.drop(index = 0)
    print(x_sum)
    k = 0
    for i in X['id']:
        if k%12==0:
            x_sum = x_sum.append(X.loc[k:k+11].sum(axis=0), ignore_index = True)
        k+=1
    return x_sum

In [243]:
X_period = sum_by_period(X)

Empty DataFrame
Columns: [id, feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7, feature_8, feature_9, feature_10, feature_11, feature_12, feature_13, feature_14, feature_15, feature_16, feature_17, feature_18, feature_19, feature_20, feature_21, feature_22, feature_23, feature_24, feature_25, feature_26, feature_27, feature_28, feature_29, feature_30, feature_31, feature_32, feature_33, feature_34, feature_35, feature_36, feature_37, feature_38, feature_39, feature_40, feature_42, feature_43, feature_44, feature_45, feature_46, feature_47, feature_48, feature_49]
Index: []

[0 rows x 49 columns]


In [244]:
X_period

,id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_39,feature_40,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,0.0,731.000000,2.866555,3523.629746,5161.610000,0.000000,5.343986,4693.534964,1772.000000,3.740000,...,188.690000,5.100000,138.431547,0.000000,718.500000,9.00000,6.009026,1714.000000,3780.140000,3.000000
1,12.0,1626.500000,0.725314,83286.958225,13432.230000,0.000000,4.207655,14398.120000,1374.000000,4792.498484,...,0.000000,19.640000,40620.522617,1.000000,1500.000000,81.00000,4.068892,1631.000000,3466.630853,2452.009568
2,24.0,7.170000,8.559721,1.696433,50.020000,0.000000,10.500171,4693.534964,122.000000,0.000000,...,0.000000,5.480000,0.000987,0.000000,7.170000,0.00000,2.566367,121.000000,29.640000,0.000000
3,36.0,971.781683,8.230128,31578.475518,3959.174433,13198.746579,7.032080,4013.201223,1446.911293,1042.956985,...,25.781989,23.543156,4063.577415,11.833557,784.539926,116.55664,5.852904,1446.667448,2765.923140,580.688209
4,48.0,1512.000000,1.604314,30819.401741,5523.990000,0.000000,12.082259,6269.390000,2731.000000,533.350000,...,6.500000,40.580000,1955.666336,2.000000,1244.800000,436.00000,9.431095,2284.000000,3466.630853,2506.932045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5101,61212.0,1130.000000,3.507360,2161.714785,14620.660000,1178.533981,5.585183,17169.430000,1715.000000,142.500000,...,1.900000,3.230000,429.471227,0.000000,1100.010000,145.00000,5.042036,2007.000000,10300.810000,78.000000
5102,61224.0,968.100000,9.068844,0.000000,2585.990000,0.000000,9.272238,1737.390000,1501.215057,21.610000,...,0.000000,10.350000,0.000000,5.000000,895.000000,102.00000,5.413837,1229.000000,2250.070000,20.000000
5103,61236.0,270.820000,9.618391,0.000000,1742.780000,0.000000,10.213948,769.100000,230.000000,486.250000,...,0.000000,32.030000,0.000000,4.000000,250.000000,67.00000,4.296799,207.000000,3466.630853,92.000000
5104,61248.0,1034.770000,0.823126,11866.732524,871.700000,0.000000,6.580039,884.640000,783.000000,67.920000,...,0.000000,6.570000,2290.858807,5.000000,1023.770000,52.00000,8.275377,731.000000,598.450000,55.000000


In [245]:
def split_X(X_period):
    x_train = X_period.drop(index = range(y_train.shape[0],X_period.shape[0]))
    x_test = X_period.drop(index = range(y_train.shape[0]))
    return x_train, x_test

In [246]:
x_train, x_test = split_X(X_period.drop('id',axis =1))

In [407]:
def shape_all_data():
    d ={'x_test shape:':x_test.shape,
       'y_test shape:':y_test.shape,
       'x_train shape:':x_train.shape,
       'y_train shape:':y_train.shape}
    for i,k in d.items():
        print(i,k,'\n')

In [248]:
shape_all_data()

x_test shape: (1022, 48) 

y_test shape: (1022, 1) 

x_train shape: (4084, 48) 

y_train shape: (4084, 1) 



In [408]:
def norm(x_test, x_train):
    norm_x_train = (x_train-x_train.mean())/x_train.std()
    norm_x_test = (x_test-x_test.mean())/x_test.std()
    return norm_x_train,norm_x_test

In [409]:
norm_x_train,norm_x_test = norm(x_test, x_train)

In [251]:
for column in norm_x_train.columns:
    norm_x_train[column] = norm_x_train[column].fillna(norm_x_train[column].mean())

In [252]:
for column in norm_x_train.columns:
    norm_x_train[column] = norm_x_train[column].fillna(norm_x_train[column].mean())

In [253]:
norm_x_train

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_39,feature_40,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,-0.342535,-0.264098,-0.277732,0.037294,-0.257892,-0.425304,0.002519,0.204511,-0.601818,0.096157,...,0.678711,-0.261334,-0.268614,-0.441248,-0.206832,-0.635418,0.299201,0.150568,0.084235,-0.635193
1,0.606794,-0.973294,0.391613,1.580414,-0.257892,-0.730625,2.225250,-0.100530,1.530241,1.670211,...,-0.141934,-0.100871,1.841437,-0.407008,1.062383,-0.310991,-0.298099,0.089015,0.003384,1.451396
2,-1.109876,1.621522,-0.307287,-0.916418,-0.257892,0.960113,0.002519,-1.060104,-0.603483,-0.880276,...,-0.141934,-0.257141,-0.275829,-0.441248,-1.362085,-0.675971,-0.760674,-1.030795,-0.882983,-0.637749
3,-0.087280,1.512358,-0.042306,-0.187054,-0.066028,0.028271,-0.153304,-0.044648,-0.139136,-0.179046,...,-0.029804,-0.057796,-0.064023,-0.036063,-0.099578,-0.150775,0.251137,-0.047685,-0.177321,-0.142995
4,0.485411,-0.682162,-0.048676,0.104907,-0.257892,1.385205,0.363452,0.939520,-0.366024,0.057409,...,-0.113664,0.130222,-0.173894,-0.372768,0.647919,1.288616,1.352738,0.573278,0.003384,1.498191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4079,-0.357843,0.481884,-0.307301,-0.670325,-0.257892,-1.048161,-0.483167,-0.768093,-0.544749,-0.653547,...,-0.141934,-0.246988,-0.275829,-0.372768,-0.342442,-0.468698,0.041666,-0.741572,-0.656356,-0.581516
4080,-0.474403,1.756625,-0.307301,-0.840832,-0.257892,-0.864568,-0.902572,-0.003028,0.009520,-0.817943,...,-0.141934,-0.005299,-0.275829,-0.270047,-0.404790,-0.315497,-0.103371,-1.004839,-0.806732,1.337812
4081,-0.632486,0.977955,-0.307301,-0.845339,-0.257892,-1.347353,-0.661854,-0.619405,-0.589160,-0.817419,...,0.392362,-0.113894,-0.275829,-0.441248,-0.676206,0.653280,0.317459,-0.840204,-0.830303,-0.623265
4082,-0.516924,3.033507,-0.303360,8.270739,-0.257892,-1.631779,2.312812,-0.003028,-0.548511,8.482330,...,-0.141934,1.244085,-0.275576,-0.441248,-0.463436,-0.563323,-0.653345,-0.355201,7.757162,-0.589184


In [353]:
class model(object):
    """
    Parameters:
    -----------
    C: str
    C it is type of classification
        'KNN' : K-Neighbors Classifier
        'DTC' : Decision Tree Classifier
        'RFC' : Random Forest Classifier
    """    
    def __init__(self, C):
        
        self.C = C
        d = {'KNN' : KNeighborsClassifier(),
        'DTC' : DecisionTreeClassifier(),
        'RFC' : RandomForestClassifier()}
      
        params = {'KNN' : {'knn__n_neighbors': range(3, 40)},
                'DTC' : {'max_depth': range(3, 40), 'max_features':[.76]},
                'RFC' :  {'max_depth': range(33,40), 'max_features': [.76]}}
    
        self.classifier = d[self.C]
        self.param = params[self.C]
        
    def fit(self, norm_x_train, norm_x_test, y_train):
        """
        Main training function
        
        """
        self.x_test = norm_x_test
        self.x_train = norm_x_train
        self.y_train = y_train         
        
        ### Train
        if self.C =='KNN':
            knn_pipe = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier(n_jobs=-1))])
            self.grid = GridSearchCV(knn_pipe, self.param, cv=5, n_jobs=-1)
            self.grid.fit(self.x_train, self.y_train)
        else:
            self.grid = GridSearchCV(self.classifier, self.param, cv=5, n_jobs=-1)
            self.grid.fit(self.x_train, self.y_train)
            
        best_score, best_parm = self.grid.best_score_, self.grid.best_params_
        return best_score, best_parm
        
    def predict(self, norm_x_test):
        """
        prediction
        
        """        
        prediction =self.grid.predict(norm_x_test)
        print(prediction)
        y_test['score'] = prediction
        return y_test 

### K-Neighbors Classifier

In [354]:
knn = model('KNN')

In [355]:
%%time
knn.fit(norm_x_train, norm_x_test, y_train)

CPU times: user 1.05 s, sys: 40.1 ms, total: 1.09 s
Wall time: 29.1 s


/home/andrii/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:335: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


(0.7485288117695058, {'knn__n_neighbors': 33})

In [357]:
y_test_knn = knn.predict(norm_x_test)

In [359]:
y_test_knn.score.sum()

48

### Decision Tree Classifier

In [361]:
dtc = model('DTC')

In [362]:
%%time
dtc.fit(norm_x_train, norm_x_test, y_train)

CPU times: user 644 ms, sys: 32.2 ms, total: 676 ms
Wall time: 10.4 s


(0.7458390332877337, {'max_depth': 4, 'max_features': 0.76})

In [363]:
y_test_dtc = dtc.predict(norm_x_test)

In [364]:
y_test_dtc.score.sum()

129

### Random Forest Classifier

In [376]:
rfc = model('RFC')

In [377]:
%%time
rfc.fit(norm_x_train, norm_x_test, y_train)

/home/andrii/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


CPU times: user 13.1 s, sys: 52 ms, total: 13.1 s
Wall time: 2min 33s


(0.7629799961600308, {'max_depth': 38, 'max_features': 0.76})

In [378]:
y_test_rfc = rfc.predict(norm_x_test)

In [380]:
y_test_rfc.score.sum()

142

In [415]:
y_test_rfc

,score
0,0
1,0
2,0
3,0
4,0
...,...
1017,0
1018,0
1019,0
1020,0


In [417]:
y_test = pd.read_csv('test.csv', encoding='latin-1')

In [419]:
y_test['score'] = y_test_rfc['score']

In [421]:
y_test.to_csv('Predct_by_Fesh_for_20_hours.csv')